# ase-notebook by example

In [ ]:
from ase_notebook import AseView, get_example_atoms

In [ ]:
ase_view = AseView(
    rotations="45x,45y,45z",
    atom_font_size=16,
    axes_length=30,
    canvas_size=(400, 400),
    zoom=1.2,
    show_bonds=True
)
ase_view.config.uc_dash_pattern=(.6,.4)
ase_view.add_miller_plane(
    1, 0, 0, color="green")
ase_view.config

In [ ]:
atoms = get_example_atoms()
atoms

In [ ]:
ase_view.make_svg(atoms, center_in_uc=True)

In [ ]:
ase_view.config.canvas_color_background = "blue"
ase_view.config.canvas_background_opacity = 0.2
render = ase_view.make_render(
    atoms, center_in_uc=True)
render